In [1]:
import importlib
#importlib.reload(i24_motion_carla_cosim)
import i24_motion_carla_cosim
import matplotlib.pyplot as plt

In [12]:
cosim = i24_motion_carla_cosim.I24MotionCARLACoSim(1, -2, 1669812000+50, 1000)

Checking behind!
Checking behind!


In [3]:
ghosts = cosim.getGhostData()

In [4]:
visible = cosim.getVisibleData()

In [5]:
hero_state = cosim.getHeroData()

In [6]:
cosim.tick(hero_state, visible)

In [7]:
visible[-3][58907]

{'id': 58907,
 'class': 4,
 'length': 22.72879870605469,
 'width': 3.0518698219299316,
 'time': 1669812050.0000005,
 's': 1121.9994156403754,
 't': -10.057062908000022,
 'velocity': 29.763865550972522,
 'lane': -3}

In [8]:
cosim.current_timestamp

1669812050.03

In [9]:
cosim.getCurrentBehindGhostWindow()

(1669812050.03, 1669812050.28, 787.024044909006, 837.024044909006)

In [10]:
cosim.getCurrentAheadGhostWindow()

(1669812050.03, 1669812050.28, 1137.024044909006, 1187.024044909006)

In [11]:
cosim.getHeroData()

{'id': 59002,
 'class': 0,
 'length': 5.353098834228516,
 'width': 1.6105653556823731,
 'time': 1669812050.03,
 's': 987.024044909006,
 't': -4.452539498231484,
 'velocity': 35.3289773877256,
 'lane': -2}

In [3]:
for i in range(1000):
    print(i)
    hero_state = cosim.getHeroData()
    visible = cosim.getVisibleData()
    cosim.tick(hero_state, visible)

0
Checking behind!
Checking behind!
1
Checking behind!
Checking behind!
2
Checking behind!
Checking behind!
3
Checking behind!
Checking behind!
4
Checking behind!
Checking behind!
5
Checking behind!
Checking behind!
6
Checking behind!
Checking behind!
7
Checking behind!
8
Checking behind!
9
Checking behind!
10
Checking behind!
11
Checking behind!
12
Checking behind!
13
Checking behind!
Checking behind!
14
Checking behind!
Checking behind!
15
Checking behind!
Checking behind!
Checking behind!
16
Checking behind!
Checking behind!
17
Checking behind!
Checking behind!
18
Checking behind!
Checking behind!
19
Checking behind!
Checking behind!
20
Checking behind!
Checking behind!
21
Checking behind!
Checking behind!
22
Checking behind!
Checking behind!
23
Checking behind!
Checking behind!
24
Checking behind!
Checking behind!
25
Checking behind!
Checking behind!
Checking behind!
26
Checking behind!
Checking behind!
Checking behind!
27
Checking behind!
Checking behind!
Checking behind!
28
Check

In [4]:
cosim.getGhostData()

{'behind': {-1: {},
  -2: {61609: {'id': 61609,
    'class': 0,
    'length': 5.174113053894044,
    'width': 1.5926968242645265,
    'time': 1669812100.0,
    's': 817.9382087220257,
    't': -4.248620744316022,
    'velocity': 34.37166200021395,
    'lane': -2}},
  -3: {},
  -4: {61529: {'id': 61529,
    'class': 1,
    'length': 4.700773099136353,
    'width': 1.7397997638702394,
    'time': 1669812100.0,
    's': 834.1501555614858,
    't': -12.806899494472821,
    'velocity': 26.319941593990578,
    'lane': -4}}},
 'ahead': {-1: {},
  -2: {61507: {'id': 61507,
    'class': 2,
    'length': 5.80240556602478,
    'width': 2.075187504386902,
    'time': 1669812099.98,
    's': 1153.3633839858621,
    't': -5.742207977817188,
    'velocity': 35.65363602722032,
    'lane': -2}},
  -3: {},
  -4: {}}}

In [15]:
len(cosim.getVisibleData()[-4])

6

In [5]:
cosim.getVisibleIdsFlat()

[58999,
 58965,
 59178,
 58976,
 59143,
 59188,
 59041,
 59158,
 59115,
 59033,
 59214,
 59258,
 59187,
 59700,
 59077,
 58943,
 58944,
 58907,
 59389,
 58985,
 58934,
 58929,
 59180,
 58925]

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from moviepy.video.VideoClip import VideoClip

def get_numpy_array_from_figure(fig):
    # Draw the canvas, cache the renderer
    fig.canvas.draw()
    
    # Get the RGBA buffer from the figure
    w, h = fig.canvas.get_width_height()
    # Use tostring_rgb() for RGB or tostring_argb() for ARGB (need to roll axis)
    buf = np.frombuffer(bytes(fig.canvas.get_renderer().buffer_rgba()), dtype='uint8')
    
    # Reshape the buffer to a 3D array (H, W, 3)
    image_np = buf.reshape(h, w, 4)
    image_np = image_np[:, :, :3]
    return image_np

def compute_world_bounds(cosim, padding=10.0):
    return cosim.hero_state["s"] - cosim.visible_window - cosim.ghost_window - padding, cosim.hero_state["s"] + cosim.visible_window + cosim.ghost_window + padding, -50, 0


def bbox_corners_2d(vehicle_data):
    cx, cy = vehicle_data["s"], vehicle_data["t"]
    length, width = vehicle_data["length"], vehicle_data["width"] / 2.0
    return np.array([[cx + length, cy + width], [cx + length, cy - width], [cx, cy - width], [cx, cy + width]])


def make_video_from_snapshots(cosim, snapshots, fps=20, out_path="carla_bboxes.mp4"):
    xmin, xmax, ymin, ymax = compute_world_bounds(cosim)

    duration = len(snapshots) / fps

    def make_frame(t):
        frame_idx = int(t * fps)
        frame_idx = min(frame_idx, len(snapshots) - 1)
        frame = snapshots[frame_idx]

        fig, ax = plt.subplots(figsize=(18, 6))
        ax.set_xlim(xmin, xmax)
        ax.set_ylim(ymin, ymax)
        ax.set_aspect("equal")
        ax.set_title(f"Frame {frame_idx}")

        # Draw each vehicle bbox
        for car in frame:
            corners = bbox_corners_2d(car)
            poly = plt.Polygon(corners, fill=False, linewidth=1.0)
            ax.add_patch(poly)
            ax.text(
                car["s"] + 0.5, car["t"],
                str(car["id"]),
                color="red",
                fontsize=8,
                ha="center", va="center",
            )

        # Optional: draw axes labels / grid
        ax.set_xlabel("X (m)")
        ax.set_ylabel("Y (m)")
        ax.grid(True, linewidth=0.2)

        # Plot ghost and visible regions
        #Behind ghost region
        ax.axvspan(cosim.hero_state["s"] - cosim.visible_window - cosim.ghost_window, cosim.hero_state["s"] - cosim.visible_window, ymin=ymin, ymax=ymax, color='yellow', alpha=0.7)

        #Visible region
        ax.axvspan(cosim.hero_state["s"] - cosim.visible_window, cosim.hero_state["s"] + cosim.visible_window, ymin=ymin, ymax=ymax, color='green', alpha=0.7)

        #Ahead ghost region
        ax.axvspan(cosim.hero_state["s"] + cosim.visible_window, cosim.hero_state["s"] + cosim.visible_window + cosim.ghost_window, ymin=ymin, ymax=ymax, color='yellow', alpha=0.7)

        fig.tight_layout()
        image = get_numpy_array_from_figure(fig)
        plt.close(fig)
        return image

    clip = VideoClip(make_frame, duration=duration)
    clip.write_videofile(out_path, fps=fps, codec="libx264")

def tickAndGetVisibleAndGhostSnapshots(cosim):
    vehicles = []
    hero_state = cosim.getHeroData()
    visible_state = cosim.getVisibleData()
    ghost_state = cosim.getGhostData()
    cosim.tick(hero_state, visible_state) # Change nothing for now
    vehicles.append(hero_state)
    for lane in visible_state:
        for id in visible_state[lane]:
            vehicles.append(visible_state[lane][id])

    for position in ghost_state:
        for lane in ghost_state[position]:
            for id in ghost_state[position][lane]:
                vehicles.append(ghost_state[position][lane][id])
    return vehicles

def collectXTimestepsFromCosim(cosim, steps=40):
    result = []
    for i in range(steps):
        result.append(tickAndGetVisibleAndGhostSnapshots(cosim))
        print(i)
    return result

if __name__ == "__main__":
    cosim = i24_motion_carla_cosim.I24MotionCARLACoSim(1, -2, 1669812000+50, 1000)
    snapshots = collectXTimestepsFromCosim(cosim)

    # 2) Make top-down bbox video
    make_video_from_snapshots(cosim, snapshots, fps=20, out_path="carla_bboxes.mp4")


Checking behind!
Checking behind!
Checking behind!
Checking behind!
0
Checking behind!
Checking behind!
1
Checking behind!
Checking behind!
2
Checking behind!
Checking behind!
3
Checking behind!
Checking behind!
4
Checking behind!
Checking behind!
5
Checking behind!
Checking behind!
6
Checking behind!
7
Checking behind!
8
Checking behind!
9
Checking behind!
10
Checking behind!
11
Checking behind!
12
Checking behind!
Checking behind!
13
Checking behind!
Checking behind!
14
Checking behind!
Checking behind!
Checking behind!
15
Checking behind!
Checking behind!
16
Checking behind!
Checking behind!
17
Checking behind!
Checking behind!
18
Checking behind!
Checking behind!
19
Checking behind!
Checking behind!
20
Checking behind!
Checking behind!
21
Checking behind!
Checking behind!
22
Checking behind!
Checking behind!
23
Checking behind!
Checking behind!
24
Checking behind!
Checking behind!
Checking behind!
25
Checking behind!
Checking behind!
Checking behind!
26
Checking behind!
Checking be

MoviePy - Done !
MoviePy - video ready carla_bboxes.mp4


In [15]:
cosim.hero_state

{'id': 59002,
 'class': 0,
 'length': 5.353098834228516,
 'width': 1.6105653556823731,
 'time': 1669812059.9799905,
 's': 987.024044909006,
 't': -4.452539498231484,
 'velocity': 35.3289773877256,
 'lane': -2}